In [1]:
"""

From Dima Pasechnik:

An obvious way to find invariant forms is by linear algebra in the representation \rho(g). Take a matrix of unknowns U, and for each generator g in G, write \rho(g)U = \lambda_g*U*\rho(g)^{-1 T}, where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}^T. In the solution space pick a nonzero element.

EDIT: nontrivial \lambda_g arise when one has sesquilinear forms (corrected above). Also note that a convenient way to form the system of equations is to take the Kroneck products \rho(g)U \otimes \lambda_g*U*\bar{\rho(g)}^{-1 T}. Then U can be recovered from a common, for all the generators, eigenvector with eigenvalue 1, of these Kronecker products, such an eigenvector gives you an embedding into the full unitary group of the corresponding form - I don't recall off the top of my head whether it will be unique for the irreducible representation \rho in positive characteristic.

NOTE: the theorem (https://mathoverflow.net/questions/271932/formula-for-the-frobenius-schur-indicator-of-a-finite-group) only guarantees a bilinear form, not a sesqilinear form.

""";

In [2]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(GF(q**2),[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [5]:
def invariant_symmetric_sesquilinear_form(q,n,partition):
    """
    Computes the matrix of a S_n-invariant symmetric sesquilinear form.

    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 

    The equations are \rho(g)U = \lambda_g*U*\rho(g)^{-1 T} where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}^T.

    The variables for U can be extracted to yield a matrix over GF(q^2) for each g. 
    
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.

    Note: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T}
    
    """

    # Define the group G and its representation as a Specht module
    SGA = SymmetricGroupAlgebra(GF(q^2), n)
    SM = SGA.specht_module(partition)
    G = SGA.group()
    
    # Get representation from Specht module
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    
    # Initialize U as a matrix of variables over GF(q^2)
    R = PolynomialRing(GF(q^2), 'u', d_rho^2)
    U_vars = R.gens()  # List of variable generators for U
    U = Matrix(R, d_rho, d_rho, U_vars)  # U is a d_rho x d_rho matrix of variables
    
    # for each generator of G, form the augmented system 
    def augmented_matrix(g):
    
        rho_g = rho(Permutation(g))
        rho_g_conj_inv_T = conjugate_pos_char(rho_g.inverse().transpose())
    
        # Compute lambda_g
        det_rho_g = det(rho_g)
        lambda_g = det_rho_g * (det_rho_g ** q)
    
        # Form the matrix equation
        equation_matrix = rho_g * U - lambda_g * U * rho_g_conj_inv_T
    
        # Initialize a list to hold rows of the augmented system
        augmented_system = []
    
        # Extract coefficients for each linear equation in the matrix
        for i in range(d_rho):
            for j in range(d_rho):
                # Get the (i, j) entry of the equation matrix, which is a linear combination of the u variables
                linear_expression = equation_matrix[i, j]
            
                # Extract the coefficients of each u_k in the linear expression
                row = [linear_expression.coefficient(u) for u in U_vars]
            
                # Append the row to the augmented system
                augmented_system.append(row)
    
        # Convert the augmented system to a matrix
        A = Matrix(GF(q^2), augmented_system)
    
        return A
    
    total_system = matrix(GF(q^2),0,d_rho^2)
    for g in G:
        total_system = total_system.stack(augmented_matrix(g))
    
    #compute the null space of the overall matrix
    null_space = total_system.right_kernel()
    
    #form a d_rho x d_rho matrix over GF(q^2) from the 1 dim'l null space given as vector
    U_mat = matrix(GF(q^2),d_rho,d_rho,null_space.basis()[0])
    
    return U_mat

In [4]:
invariant_symmetric_sesquilinear_form(q,n,partition)

[0 1 2 1 2 0]
[1 0 1 1 0 2]
[2 1 0 0 1 2]
[1 1 0 0 1 1]
[2 0 1 1 0 1]
[0 2 2 1 1 0]

In [103]:
def sesquilinear_form(x,y,U):
    return x*U*y.transpose()

In [104]:
R_xy = PolynomialRing(GF(q^2), d_rho, var_array='x,y')

In [105]:
x = matrix([R_xy.gens()[2*i] for i in range(d_rho)]); x

[x0 x1 x2 x3 x4 x5]

In [108]:
y = matrix([R_xy.gens()[2*i+1] for i in range(d_rho)]); y

[y0 y1 y2 y3 y4 y5]

In [109]:
U_form = matrix(R_xy,d_rho,d_rho,null_space.basis()[0]); U_form

[ 0  1 -1  1 -1  0]
[ 1  0  1  1  0 -1]
[-1  1  0  0  1 -1]
[ 1  1  0  0  1  1]
[-1  0  1  1  0  1]
[ 0 -1 -1  1  1  0]

In [110]:
#check symmetric property
sesquilinear_form(x,y,U_form) == sesquilinear_form(y,x,U_form)

True

In [115]:
#check invariance property
sesquilinear_form((rho(G[8])*x.transpose()).transpose(),(rho(G[8])*y.transpose()).transpose(),U_form) == sesquilinear_form(x,y,U_form)

True